In [1]:
import argparse, os, sys, glob

sys.path.extend([
    '/workspace/Dreambooth-SD-optimized',
])

import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid, save_image
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler


def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model



In [2]:
parser = argparse.ArgumentParser()
opt = parser.parse_args(args=[])
opt.prompt = 'a photo of a cat'
opt.precision='autocast'
opt.seed=42
opt.ckpt='/workspace/mnt/models/sd-v1-4.ckpt'
opt.config='/workspace/Dreambooth-SD-optimized/configs/stable-diffusion/v1-inference.yaml'
opt.scale=10.
opt.n_rows=0
opt.n_samples=8
opt.f=8
opt.C=4
opt.W=512
opt.H=512
opt.n_iter=25
opt.ddim_eta=0.0
opt.fixed_code=False
opt.laion400m=False
opt.plms=False
opt.ddim_steps=50
opt.skip_save=False
opt.skip_grid=True
opt.outdir='/workspace/mnt/outputs/regularization_images/'
opt.from_file=False

if opt.laion400m:
    print("Falling back to LAION 400M model...")
    opt.config = "configs/latent-diffusion/txt2img-1p4B-eval.yaml"
    opt.ckpt = "models/ldm/text2img-large/model.ckpt"
    opt.outdir = "outputs/txt2img-samples-laion400m"

seed_everything(opt.seed)

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")
#model.embedding_manager.load(opt.embedding_path)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

if opt.plms:
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

batch_size = opt.n_samples
n_rows = opt.n_rows if opt.n_rows > 0 else batch_size
if not opt.from_file:
    prompt = opt.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]

else:
    print(f"reading prompts from {opt.from_file}")
    with open(opt.from_file, "r") as f:
        data = f.read().splitlines()
        data = list(chunk(data, batch_size))

sample_path = os.path.join(outpath, "samples")
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(outpath)) - 1

start_code = None
if opt.fixed_code:
    start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

precision_scope = autocast if opt.precision=="autocast" else nullcontext
with torch.no_grad():
    with precision_scope("cuda"):
        with model.ema_scope():
            tic = time.time()
            all_samples = list()
            for n in trange(opt.n_iter, desc="Sampling"):
                opt.seed+=1
                for prompts in tqdm(data, desc="data"):
                    uc = None
                    if opt.scale != 1.0:
                        uc = model.get_learned_conditioning(batch_size * [""])
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    c = model.get_learned_conditioning(prompts)
                    shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                    samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                     conditioning=c,
                                                     batch_size=opt.n_samples,
                                                     shape=shape,
                                                     verbose=False,
                                                     unconditional_guidance_scale=opt.scale,
                                                     unconditional_conditioning=uc,
                                                     eta=opt.ddim_eta,
                                                     x_T=start_code)

                    x_samples_ddim = model.decode_first_stage(samples_ddim)
                    x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

                    if not opt.skip_save:
                        for x_sample in x_samples_ddim:
                            x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                            Image.fromarray(x_sample.astype(np.uint8)).save(
                                os.path.join(sample_path, f"{base_count:05}.jpg"))
                            base_count += 1

                    if not opt.skip_grid:
                        all_samples.append(x_samples_ddim)

            if not opt.skip_grid:
                # additionally, save as grid
                grid = torch.stack(all_samples, 0)
                grid = rearrange(grid, 'n b c h w -> (n b) c h w')

                for i in range(grid.size(0)):
                    save_image(grid[i, :, :, :], os.path.join(outpath,opt.prompt+'_{}.png'.format(i)))
                grid = make_grid(grid, nrow=n_rows)

                # to image
                grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                Image.fromarray(grid.astype(np.uint8)).save(os.path.join(outpath, f'{prompt.replace(" ", "-")}-{grid_count:04}.jpg'))
                grid_count += 1



            toc = time.time()

print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
      f" \nEnjoy.")




Global seed set to 42


Loading model from /workspace/mnt/models/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.17.mlp.fc2.weight', 'vision_model.encoder.layers.8.mlp.fc1.weight', 'vision_model.encoder.layers.16.self_attn.q_proj.weight', 'vision_model.encoder.layers.20.self_attn.v_proj.weight', 'vision_model.encoder.layers.0.self_attn.k_proj.bias', 'vision_model.encoder.layers.6.layer_norm1.weight', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.17.self_attn.k_proj.weight', 'vision_model.encoder.layers.23.mlp.fc1.bias', 'vision_model.encoder.layers.10.self_attn.k_proj.weight', 'vision_model.encoder.layers.15.mlp.fc1.bias', 'vision_model.encoder.layers.15.layer_norm2.bias', 'text_projection.weight', 'vision_model.encoder.layers.1.self_attn.k_proj.bias', 'vision_model.encoder.layers.16.mlp.fc1.bias', 'vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encoder.la

data:   0%|                                                                                                                                                                          | 0/1 [00:00<?, ?it/s]

DDIM Sampler:   0%|                                                                                                                                                                 | 0/50 [00:00<?, ?it/s]

Data shape for DDIM sampling is (8, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


...

DDIM Sampler:  78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                 | 39/50 [00:23<00:06,  1.66it/s]

DDIM Sampler:  80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 40/50 [00:24<00:06,  1.66it/s]

DDIM Sampler:  82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                           | 41/50 [00:24<00:05,  1.66it/s]

DDIM Sampler:  84%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                        | 42/50 [00:25<00:04,  1.66it/s]

DDIM Sampler:  86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 43/50 [

Data shape for DDIM sampling is (8, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps




DDIM Sampler:   2%|███                                                                                                                                                      | 1/50 [00:00<00:29,  1.67it/s]

DDIM Sampler:   4%|██████                                                                                                                                                   | 2/50 [00:01<00:28,  1.67it/s]

DDIM Sampler:   6%|█████████▏                                                                                                                                               | 3/50 [00:01<00:28,  1.66it/s]

DDIM Sampler:   8%|████████████▏                                                                                                                                            | 4/50 [00:02<00:27,  1.66it/s]

DDIM Sampler:  10%|███████████████▎                                                                                                                                         | 5/50

DDIM Sampler:  78%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                 | 39/50 [00:23<00:06,  1.66it/s]

DDIM Sampler:  80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 40/50 [00:24<00:06,  1.66it/s]

DDIM Sampler:  82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                           | 41/50 [00:24<00:05,  1.66it/s]

DDIM Sampler:  84%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                        | 42/50 [00:25<00:04,  1.66it/s]

DDIM Sampler:  86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                     | 43/50 [

Your samples are ready and waiting for you here: 
/workspace/mnt/outputs/regularization_images/ 
 
Enjoy.


In [3]:
config

{'model': {'base_learning_rate': 0.0001, 'target': 'ldm.models.diffusion.ddpm.LatentDiffusion', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'personalization_config': {'target': 'ldm.modules.embedding_manager.EmbeddingManager', 'params': {'placeholder_strings': ['*'], 'initializer_words': ['sculpture'], 'per_image_tokens': False, 'num_vectors_per_token': 1, 'progressive_words': False}}, 'unet_config': {'target': 'ldm.modules.diffusionmodules.openaimodel.UNetModel', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'tra